# MTH 652: Advanced Numerical Analysis

## Lecture 3

### Topics

* Inhomogeneous boundary conditions
* General second-order elliptic operators
* Non-symmetric variational problems

#### Inhomogeneous boundary conditions

Suppose that we now consider the problem
$$
\left\{
\begin{aligned}
    -\Delta u &= f \qquad\text{in $\Omega$} \\
    u &= g_D \qquad\text{on $\Gamma_D \subseteq \partial\Omega$} \\
    \frac{\partial u}{\partial \boldsymbol n} &= g_N \qquad\text{on $\Gamma_N = \partial\Omega \setminus \Gamma_D$}
\end{aligned}
\right.
$$

$g_N \in L^2(\Gamma_N)$, and, for simplicitly, we assume that $g_D \in H^1(\Omega)$.
Then, we look for solutions $u$ that satisfy $u - g_D \in V$ such that, for all $v \in V$,
$$
   a(u, v) = (f, v) + \int_{\Gamma_N} g_N v \, ds.
$$

The boundary term comes from integrating $-\Delta u$ by parts and substituting the Neumann boundary condition.

* Dirichlet ("essential") boundary conditions are enforced strongly
* Neumann boundary conditions are enforced by modifying the right-hand side

In [1]:
import mfem.ser as mfem
from glvis import glvis
import numpy as np

In [2]:
# Create a mesh of a square domain
mesh = mfem.Mesh(40, 40, "TRIANGLE")

The four sides of the square are tagged with different "attributes":

* The bottom side of the square has attribute 1
* The right side of the square has attribute 2
* The top side of the square has attribute 3
* The left side of the square has attribute 4

In [3]:
dirichlet_attributes = mfem.intArray([1, 0, 0, 1])
neumann_attributes = mfem.intArray([0, 1, 1, 0])

# Create finite element function space
fec = mfem.H1_FECollection(3, mesh.Dimension())
fes = mfem.FiniteElementSpace(mesh, fec)      

class DiffusionCoefficient(mfem.MatrixPyCoefficient):
    def __init__(self):
       mfem.MatrixPyCoefficient.__init__(self, 2)
    def EvalValue(self, x):
        theta = 55.0/180.0*np.pi
        ct = np.cos(theta)
        st = np.sin(theta)
        rot = np.array([[ct, -st], [st, ct]])
        # return np.array([[1.0, 0.0], [0.0, 10.0]])
        return rot.T @ np.array([[1.0, 0.0], [0.0, 10.0]]) @ rot

# define Bilinear and Linear operator
a = mfem.BilinearForm(fes)
a.AddDomainIntegrator(mfem.DiffusionIntegrator(DiffusionCoefficient()))
a.Assemble()

b = mfem.LinearForm(fes)
b.AddDomainIntegrator(mfem.DomainLFIntegrator(mfem.ConstantCoefficient(1.0)))

# To enforce inhomogeneous Neumann conditions, we need to add a boundary
# integrator to the right-hand side

# Set the Dirichlet condition
class NeumannCondition(mfem.PyCoefficient):
    def EvalValue(self, x):
        return 4*np.sin(10*np.pi*x[0]) + 2*np.cos(10*np.pi*x[1])

b.AddBoundaryIntegrator(mfem.BoundaryLFIntegrator(NeumannCondition()),
                        neumann_attributes)

b.Assemble()

# Enforce Dirichlet (essential) BCs on the bottom and left
ess_tdof_list = mfem.intArray()
fes.GetEssentialTrueDofs(dirichlet_attributes, ess_tdof_list)

# Set the Dirichlet condition
class DirichletCondition(mfem.PyCoefficient):
    def EvalValue(self, x):
        return np.sin(2*np.pi*x[0]) + np.cos(2*np.pi*x[1])

u = mfem.GridFunction(fes)
u.ProjectCoefficient(DirichletCondition())

# form linear equation (AX=B)
A = mfem.OperatorPtr()
B = mfem.Vector()
X = mfem.Vector()
a.FormLinearSystem(ess_tdof_list, u, b, A, X, B);
print("Size of linear system: " + str(A.Height()))

# solve it using PCG solver and store the solution to x
AA = mfem.OperatorHandle2SparseMatrix(A)
M = mfem.GSSmoother(AA)

X.Assign(0.0)
mfem.PCG(AA, M, B, X, 0, 1000, 1e-12, 0.0)
a.RecoverFEMSolution(X, b, u)

Size of linear system: 14641


In [4]:
glvis((mesh, u))

glvis(data_str='MFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see mesh/geom.hpp):\n#\n# POINT       = 0\n# SEGME…

#### General second-order elliptic operators

Consider a symmetric matrix of coefficients $A = (a_{ij}(\boldsymbol x))_{i,j=1}^d$.
This matrix is said to be **uniformly elliptic** if it is positive definite, i.e.
$$
   \sum_{i,j = 1}^d a_{ij}(\boldsymbol x) \xi_i \xi_j \geq \alpha \sum_{i=1}^d \xi_i^2
$$
for almost all $\boldsymbol{x} \in \Omega$, with $\alpha > 0$.

The **elliptic operator** associated with this coefficient is
$$
   Lu(x) = - \sum_{i,j=1}^d \frac{\partial}{\partial x_j} \left( a_{ij}(\boldsymbol{x}) \frac{\partial u}{\partial x_i} (\boldsymbol{x}) \right)
$$

For example, if $A$ is the identity matrix, we recover the standard Laplacian.

The bilinear form associated with this operator is given naturally by
$$
   a(u, v) = (A \nabla u, \nabla v),
$$
and by symmetry of $A$, $a(\cdot, \cdot)$ is a symmetric bilinear form.

Positive-definiteness of the coefficients implies that
$$
   a(v, v) \geq \alpha | v |_{H^1(\Omega)}^2
$$

### Non-symmetric problems
#### 2.6, 2.7, 2.8

Suppose that $H$ is a Hilbert space, and $V \subseteq H$ is a (closed) subspace

Suppose we have a bilinear form $a(\cdot, \cdot)$ such that:

* $a(\cdot, \cdot)$ is bounded (i.e. continuous) on $V$, and
* $a(\cdot, \cdot)$ is coercive on $V$.
* But, $a(\cdot, \cdot)$ is **not** necessarily symmetric

We can, of course, define the variational problem just as before:

> Given $F \in V'$, find $u \in V$ such that, for all $v \in V$
> $$ a(u, v) = F(v) $$

And, as before, we can define the Galerkin approximation problem.
Given a **finite-dimensional** subspace $V_h \subseteq V$, we seek $u_h \in V_h$ such that, for all $v_h in V_h$
$$ a(u_h, v_h) = F(v_h) $$

In the symmetric case, we understand how this works:

* There exist unique solutions $u$ and $u_h$ (Riesz Representation Theorem)
* The error $u - u_h$ is optimal in the energy norm, and quasi-optimal in the $V$ norm (depending on the constants of continuity and coercivity)
* This has important applications to diffusion problems, diffusion-reaction problems, $L^2$ projection, etc.

What can we say about the **non-symmetric** case?

As a motivating example, consider the **convection-diffusion-reaction** equation
$$
   -\alpha \Delta u + \boldsymbol\beta\cdot\nabla u + \gamma u = f,
$$
where $\alpha, \gamma > 0$ and $\beta \in \mathbb{R}^d$.
The variational formulation for this problem is:

$$
   a(u, v) := \int_\Omega \alpha \nabla u \cdot \nabla v \, dx
      + \int_\Omega (\beta \cdot \nabla u) v \, dx
      + \int_\Omega \gamma u v \, dx
      = \int_\Omega f v \, dx
$$

It is straightforward to see that $a(\cdot, \cdot)$ is continuous in $H^1$.

We can also see that $a(\cdot, \cdot)$ is **coercive** as long as the magnitude of $\beta$ is sufficiently small relative to $\alpha$ and $\gamma$.

Our goal now is to prove existence and uniqueness of the solution to the variational problem
$$ a(u, v) = F(v) \qquad\text{for all $v \in V$} $$

First, we recall the contraction mapping principle:

> **_LEMMA (Contraction Mapping):_** Let $V$ be a Banach space, and $T : V \to V$ satisfying
> $$ \| T v_1 - T v_2 \| \leq M \| v_1 - v_2 \| $$
> for all $v_1, v_2 \in V$, and $0 \leq M < 1$.
>
> Then, there exists a unique **fixed point** $u$ such that $u = T u$.

_Proof._
Uniqueness.
Suppose $T v_1 = v_1$ and $T v_2 = v_2$.
Then,
$$
   \| v_1 - v_2 \| = \| T v_1 - T v_2 \| \leq M \| v_1 - v_2 \|,
$$
(by the contraction mapping property), but $M < 1$, so we must have $\| v_1 - v_2 \| = 0$, and the fixed point is unique.

Existence.
Pick any $v_0 \in V$, and define
$$
   v_i = T^i v_0 = T \circ \cdots \circ T v_0.
$$
Note that $\| v_{k+1} - v_{k} \| = \| T v_k - T v_{k-1} \| \leq M \| v_k - v_{k-1} \|$, and so
$$
   \| v_{k+1} - v_k \| \leq M^k \| v_1 - v_0 \| \to 0
$$
(Note that for arbitrary $i, j \geq N$ for some $N$, the difference $\| v_i - v_j \|$ can be estimated using a telescoping argument and geometric series $\sum_{k=0}^\infty a r^k  = a / (1 - r)$), set $a = M^N, r = M$ (where $i$ is the smaller of $i$ and $j$).
So, this sequence is Cauchy, and a limit $v$ exists,
$$
   v = \lim_{k \to \infty} v_k = \lim_{k \to \infty} T v_{k-1} = T \lim_{k \to \infty} v_{k-1} = T v,
$$
using continuity, so $v$ is a fixed point.

> **_THEOREM (Lax-Milgram):_** Let $V$ be a Hilbert space with inner product $(\cdot, \cdot)$, and let $a(\cdot, \cdot)$ be a continuous, coercive bilinear form (not necessarily symmetric), and let $F \in V'$ be a continuous linear functional. Then, there exists a unique $u \in V$ such that, for all $v \in V$,
> $$ a(u, v) = F(v).$$


> _Proof._
>
> In the proof, we will use the constants $\alpha$ and $C$:
> $$
>  \begin{aligned}
>     a(u, v) &\leq C \| u \| \| v \| \\
>     a(u, u) &\geq \alpha \| u \|^2 \\
>  \end{aligned}
> $$
>
> For any $u \in V$, let $A_u (v) = a(u, v)$. Note that $A_u$ is a continuous, linear functional ($A_u \in V'$).
> 
> Let $A : V \to V'$ denote the map $u \mapsto A_u$.
> It is easy to check that $A$ is linear.
> We can check that $A$ is also bounded (the dual norm of $A_u$ is bounded by $C \| u \|$, where $C$ is the constant of continuity of the bilinear form).
>
> By the Riesz Representation Theorem, for any $\phi \in V'$, there exists unique $\tau_\phi$ such that $(\tau_\phi, v) = \phi(v)$ for all $v \in V$.
> Let $\tau$ denote the mapping $\phi \mapsto \tau_\phi$.
> Recall that $\tau$ is a one-to-one isometry (i.e. $\| \tau_\phi \| = \| \phi \|$ for all $\phi$).
>
> Notice that our original goal of
> $$ a(u, v) = F(v) $$
> is equivalent to
> $$ A_u (v) = F(v) $$
> for all $v \in V$, or simply
> $$ A_u = F $$
> in $V'$.
>
> Since the mapping $\tau$ is one-to-one, this is equivalent to finding $u$ such that
> $$ \tau_{A_u} = \tau_F. $$
>
> Define the operator $T : V \to V$ by
> $$ T v = v - \rho(\tau_{A_v} - \tau_F) $$
> for some $\rho \in \mathbb{R}$, $\rho \neq 0$, to be determined.
>
> **If** $T$ is a contraction mapping, then there exists a unique fixed point $u$, such that
> $$
>  u = u - \rho (\tau_{A_u} - \tau_F),
> $$
> and we have $\tau_{A_u} - \tau_F = 0$, so $u$ is the unique solution.
>
> Now, we show that there exists $\rho \neq 0$ such that $T$ is a contraction.
> Let $v_1, v_2 \in V$, and let $v = v_1 - v_2$.
> Then,
> $$
>  \begin{aligned}
>     \| T v_1 - T v_2 \|^2
>     &= \| v_1 - v_2 - \rho(\tau_{A_{v_1}} - \tau_{A_{v_2}}) \|^2 \\
>     &= \| v - \rho \tau_{A_{v}} \|^2 \\
>     &= \| v \|^2 - 2 \rho (\tau_{A_v}, v) + \rho^2 \| \tau_{A_v} \|^2 \\
>     &= \| v \|^2 - 2 \rho A_v(v) + \rho^2 A_v(\tau_{A_v}) \\
>     &= \| v \|^2 - 2 \rho a(v, v) + \rho^2 a(v, \tau_{A_v}) \\
>     &\leq \| v \|^2 - 2 \rho \alpha \| v \|^2 + \rho^2 C \| v \| \| \tau_{A_v} \| \\
>     &\leq \| v \|^2 - 2 \rho \alpha \| v \|^2 + \rho^2 C^2 \| v \| \| v \| \\
>     &= (1 - 2\rho\alpha + \rho^2 C^2) \| v \|^2 \\
>     &= (1 - 2\rho\alpha + \rho^2 C^2) \| v_1 - v_2 \|^2
>  \end{aligned}
> $$
>
> For $T$ to be a contraction, we need to choose $\rho \neq 0$ such that
> $$ 1 - 2\rho\alpha + \rho^2 C^2 < 1 $$
> i.e.
> $$ \rho(\rho C^2 - 2\alpha) < 0 $$
> i.e. take $\rho > 0$ and
> $$ \rho C^2 - 2\alpha < 0 $$
> So we can take any $\rho$ such that
> $$ 0 < \rho < 2\alpha / C^2 $$